In [215]:
# Useful packages
import os
import numpy as np
import pandas as pd
import io
import json
import datetime as pkgdt
import time
import requests 
from requests.adapters import HTTPAdapter
import itertools # list operators
from tqdm import tqdm


In [216]:
gares = pd.read_csv('ensemble_gares.csv')

In [217]:
def recherche_gare(nom_gare):
    '''Renvoie les gares dont le nom contient nom_gare'''
    gares_filtres = gares[gares['name'].str.contains(nom_gare, na=False)]
    gares_filtres.insert(1,'SNCF_ID',gares_filtres['id'].str[15:])
    
    if gares_filtres.empty:
        print('Aucune gare ne contient le nom recherché.')
        return(gares_filtres)
    else:
        return(gares_filtres)
print(recherche_gare('eijfezflpez').empty)


Aucune gare ne contient le nom recherché.
True


In [218]:
nom_recherche = 'Paris Est'

gares_recherche = recherche_gare(nom_recherche)

if gares_recherche.empty==False:        

    SNCF_ID = gares_recherche['SNCF_ID'].iloc[0]

    print('Résultats présentés pour la gare '+ gares_recherche['label'].iloc[0] + ' (ID: ' + SNCF_ID +').')

    def page_departs(numero_page) :
        return requests.get(
            ('https://api.sncf.com/v1/coverage/sncf/stop_areas/stop_area:SNCF:'+SNCF_ID+'/departures?datetime=20221027T105600').format(numero_page),
            auth=('16cbfb01-1943-471f-aac9-7a1139abab77', ''))

    ######################################
    # on commence par la première page qui nous donne le nombre de résultats par page ainsi que le nombre total de résultats

    page_initiale = page_departs(0)
    item_per_page = page_initiale.json()['pagination']['items_per_page']
    total_items = page_initiale.json()['pagination']['total_result']
    departsdfs = []

    # on fait une boucle sur toutes les pages suivantes
    print_done = {}

    for page in range(int(total_items/item_per_page)+1) :
        departs_page = page_departs(page)

        ensemble_departs = departs_page.json()

        if 'departures' not in ensemble_departs:
            # pas d'arrêt
            continue

        # on ne retient que les informations qui nous intéressent
        for departs in ensemble_departs['departures']:

            departs['label'] = departs['display_informations']['direction']
            

        departs = ensemble_departs['departures']
        dp= pd.DataFrame(departs)

        departsdfs.append(dp)
        if page % 10 == 0:
            print("je suis à la page", page, "---", dp.shape)
        
    departs_liste = pd.concat(departsdfs)
    departs_liste.head()

Résultats présentés pour la gare Paris Est (Paris) (ID: 87113001).
je suis à la page 0 --- (10, 6)


In [219]:
departs_gares = pd.merge(departs_liste,gares,on='label',how='left')
departs_gares_loc= departs_gares[['label','lat','lon']]
departs_gares_loc.head(30)

,label,lat,lon
0,Luxembourg (Luxembourg),49.599722,6.134722
1,Provins (Provins),48.555737,3.302837
2,Château-Thierry (Château-Thierry),49.038140,3.409302
3,Karlsruhe Hbf (Karlsruhe),48.993500,8.401900
4,Meaux (Meaux),48.957301,2.873823
5,Coulommiers (Coulommiers),48.807783,3.082122
6,Meaux (Meaux),48.957301,2.873823
7,Dijon (Dijon),47.323411,5.027256
8,Provins (Provins),48.555737,3.302837
9,Château-Thierry (Château-Thierry),49.038140,3.409302


In [235]:
#Affichage Folium
import folium

gares_initiale_latitude = gares_recherche['lat'].iloc[0]
gares_initiale_longitude = gares_recherche['lon'].iloc[0]


departs_gares_latitude =departs_gares_loc["lat"]
departs_gares_longitude = departs_gares_loc["lon"]


departs_gares_latitude.head()

fmap= folium.Map(location=[departs_gares_latitude.iloc[0],departs_gares_longitude.iloc[0]])

folium.Marker([gares_initiale_latitude, gares_initiale_longitude],
        popup=gares_recherche['label'].iloc[0],
        icon=folium.Icon(color='green')).add_to(fmap)

n = departs_gares_loc.shape[0] #Nombre de gares
print(departs_gares_longitude.iloc[2])

for i in range(n):
    longitude = departs_gares_longitude.iloc[i]
    latitude = departs_gares_latitude.iloc[i]
    nom = departs_gares_loc['label'].iloc[i]
    if (not(pd.isna(latitude)) and not(pd.isna(longitude))):
   

        
        folium.Marker([latitude, longitude],
        popup=nom,
        icon=folium.Icon(color='red')).add_to(fmap)
fmap.save(outfile='Cartes\Carte_SNCF_departs'+ str(gares_recherche['label'].iloc[0]) +'.html')
fmap

3.409302
